# 通过CNN分析价格曲线，预测涨跌  
 
20 x 20=400个工作日  20个月 80个礼拜  => if 未来20个工作日的平均股价 > 当前股价


In [2]:
import baostock as bs
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
import matplotlib. pyplot as plt 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Reshape,Dropout,Activation
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.layers import Conv1D,MaxPooling1D
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split

In [8]:
import datetime
import baostock as bs

In [2]:
bs.login()


login success!


In [3]:
stock_rs = bs.query_all_stock("2021-04-02")
stock_df = stock_rs.get_data()
stock_df['code'].to_csv('stock_codes.csv')

In [14]:
stock_df[160:167:]

,code,tradeStatus,code_name
160,sh.000852,1,中证1000指数
161,sh.000853,1,中证申万一带一路主题投资指数
162,sh.000854,1,中证500原材料指数
163,sh.000855,1,央视财经500指数
164,sh.000856,1,中证500工业指数
165,sh.000857,1,中证500医药卫生指数
166,sh.000858,1,中证500信息技术指数


In [21]:
with open("stock_codes.txt", "r") as f:
    stock_code_list = [line.strip() for line in f.readlines()]



In [3]:
stock_code_list = pd.read_csv('stock_codes.csv')['code']

In [4]:
stock_code_list

0    sz.000858
Name: code, dtype: object

In [29]:
stock_code = "sz.000858"
stock_info_path = "stock_info/" + stock_code + ".csv"
stock = pd.read_csv(stock_info_path, parse_dates=['date'])
stock = stock[stock['date'] < datetime.datetime.now()- datetime.timedelta(days=20)]
stock

,date,close,volume,amount,turn
0,2010-01-04,31.45,26409039,8.363519e+08,0.695795
1,2010-01-05,31.78,29098216,9.237874e+08,0.766646
2,2010-01-06,30.94,42822040,1.340543e+09,1.128226
3,2010-01-07,30.00,61705156,1.850036e+09,1.625736
4,2010-01-08,29.94,36477184,1.075206e+09,0.961059
...,...,...,...,...,...
2726,2021-03-24,260.80,17942151,4.728182e+09,0.472700
2727,2021-03-25,255.98,18625855,4.750886e+09,0.490700
2728,2021-03-26,265.95,23020269,6.054732e+09,0.606500
2729,2021-03-29,272.09,28199481,7.712654e+09,0.742900


In [51]:
stock[0:-1]

,date,close,volume,amount,turn
0,2010-01-04,31.45,26409039,8.363519e+08,0.695795
1,2010-01-05,31.78,29098216,9.237874e+08,0.766646
2,2010-01-06,30.94,42822040,1.340543e+09,1.128226
3,2010-01-07,30.00,61705156,1.850036e+09,1.625736
4,2010-01-08,29.94,36477184,1.075206e+09,0.961059
...,...,...,...,...,...
2725,2021-03-23,262.25,12695157,3.309793e+09,0.334500
2726,2021-03-24,260.80,17942151,4.728182e+09,0.472700
2727,2021-03-25,255.98,18625855,4.750886e+09,0.490700
2728,2021-03-26,265.95,23020269,6.054732e+09,0.606500


In [55]:
for i in range(20, 0, -1):
    print(i)

20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1


In [11]:
bs.login()

stock_code = "sz.000858"
stock_info_path = "stock_info/" + stock_code + ".csv"

to_date = datetime.datetime.now().strftime("%Y-%m-%d")
# to_date = '2021-04-18'          # 今日日期
re_download = True              # 重新下载
re_train = True                # 是否训练
predict_period = 20             # 预测天数
history_period = 400            # 分析天数
epoch = 200
start_date = '2010-01-01'
rs = bs.query_history_k_data(stock_code, "date, close, volume, amount, turn", start_date=start_date, end_date=to_date, frequency="d", adjustflag="3")
data_list = []
while (rs.error_code == '0') & rs.next():  # 获取一条记录，将记录合并在一起
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)
result.to_csv(stock_info_path, index=False)

login success!


In [19]:
rs